# IFT6390 Project - Data Exploration snippets

## Useful piece of code

## Setup - data

In [2]:
import pandas as pd
import numpy as np

## Sentiment140 Data Exploration

In [26]:
def printStats(dataset):
    print(f"Total of {dataset.shape[0]} text samples")
    gr_by=dataset.groupby('target').count()
    print(gr_by['text'].index)
    print(gr_by['text'].values)
    #print(gr_by)
    print('='*80)
    print(dataset.head())
    print('='*80)
    print(dataset['target'].value_counts(normalize=True))
    

    
printStats(s140)
print('='*80)
s140_pos = s140[(s140.target == 'positive')]
s140_neg = s140[(s140.target == 'negative')]
s140_neut=s140.loc[s140['target']=='neutral']
print(f"**Only {s140_neut.shape[0]} 'neutral' text samples**")

Total of 1600498 text samples
Index(['negative', 'neutral', 'positive'], dtype='object', name='target')
[800177    139 800182]
                                                text    target
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  negative
1  Got a headache :/ MC stop making music, you ca...  negative
2  lol still worked like crazy lol  . lol Your la...  negative
3  why won't netflix send me S. Darko? I know it'...  negative
4  [ToZ] Clan Website offline  http://www.theoutl...  negative
positive    0.499958
negative    0.499955
neutral     0.000087
Name: target, dtype: float64
**Only 139 'neutral' text samples**


In [28]:
printStats(cc)
print('='*80)
print('='*80)
printStats(mr)

#print('='*80)
#s140_pos = s140[(s140.target == 'positive')]
#s140_neg = s140[(s140.target == 'negative')]
#s140_neut=s140.loc[s140['target']=='neutral']
#print(f"**Only {s140_neut.shape[0]} 'neutral' text samples**")

Total of 6027 text samples
Index(['N', 'No', 'Y', 'Yes'], dtype='object', name='target')
[1041   58 2534  554]
                                                text  confidence target
0  Global warming report urges governments to act...      1.0000    Yes
1  Fighting poverty and global warming in Africa ...      1.0000    Yes
2  Carbon offsets: How a Vatican forest failed to...      0.8786    Yes
3  Carbon offsets: How a Vatican forest failed to...      1.0000    Yes
4  URUGUAY: Tools Needed for Those Most Vulnerabl...      0.8087    Yes
Y      0.605207
N      0.248627
Yes    0.132314
No     0.013852
Name: target, dtype: float64
Total of 50000 text samples
Index(['negative', 'positive'], dtype='object', name='target')
[25000 25000]
                                                text    target
0  Story of a man who has unnatural feelings for ...  negative
1  Airport '77 starts as a brand new luxury 747 p...  negative
2  This film lacked something I couldn't put my f...  negative
3  Sorr

In [32]:
cc['old_target']=cc.target
cc=cc.replace({'target': {'Yes': 'positive', 'Y': 'positive', 'No': 'negative', 'N': 'negative'}})
printStats(cc)

Total of 6027 text samples
Index(['negative', 'positive'], dtype='object', name='target')
[1099 3088]
                                                text  confidence    target  \
0  Global warming report urges governments to act...      1.0000  positive   
1  Fighting poverty and global warming in Africa ...      1.0000  positive   
2  Carbon offsets: How a Vatican forest failed to...      0.8786  positive   
3  Carbon offsets: How a Vatican forest failed to...      1.0000  positive   
4  URUGUAY: Tools Needed for Those Most Vulnerabl...      0.8087  positive   

  old_target  
0        Yes  
1        Yes  
2        Yes  
3        Yes  
4        Yes  
positive    0.737521
negative    0.262479
Name: target, dtype: float64


## Import cleaned dataframe

In [4]:
s140=pd.read_pickle('data/s140_clean_28nov.pkl')
mr=pd.read_pickle('data/mr_clean_28nov.pkl')
cc=pd.read_pickle('data/cc_clean_28nov.pkl')

In [103]:
print(s140['length'].describe().astype('int64') )
print('='*80)
print(s140.head())

count    1600498
mean           8
std            4
min            0
25%            5
50%            8
75%           11
max          118
Name: length, dtype: int64
                                                text    target  \
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  negative   
1  Got a headache :/ MC stop making music, you ca...  negative   
2  lol still worked like crazy lol  . lol Your la...  negative   
3  why won't netflix send me S. Darko? I know it'...  negative   
4  [ToZ] Clan Website offline  http://www.theoutl...  negative   

                                               lemma  length  
0  [switchfoot, _link_, awww, 's, bummer, shoulda...      11  
1  [got, headache, mc, stop, making, music, ca, n...      11  
2  [lol, still, worked, like, crazy, lol, lol, la...      18  
3  [wo, n't, netflix, send, s., darko, know, 's, ...      16  
4              [toz, clan, website, offline, _link_]       5  


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer 

def make_tfidf(df):
    tf_vect=TfidfVectorizer(use_idf=False,stop_words=stopw)
    tf=tf_vect.fit_transform(df.tolist())
    
    l=tf.shape[0]
    
    w_count=np.array(tf.sum(axis=0,))[0]/l
    #wcm=w_count.max()
    #w_count=w_count/wcm
    wcr=w_count.argsort()[::-1].argsort()

    
    idf_vect=TfidfVectorizer(use_idf=True,stop_words=stopw)
    idf=idf_vect.fit_transform(df.tolist())
    
    w_marker=np.array(idf.sum(axis=0))[0]/l
    #wmm=w_marker.max()
    #w_marker=w_marker/wmm
    wmr=w_marker.argsort()[::-1].argsort()
    
    
    # SRC -> https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/
    feature_names=tf_vect.get_feature_names()
    
    tf_idf = pd.DataFrame(np.array([w_count, w_marker, wcr, wmr]).T, index=feature_names, columns=["tf", "idf", "rank_tf", "rank_idf"])
    tf_idf = tf_idf.sort_values(by=["tf"],ascending=False)

    tf_idf.rank_tf=tf_idf.rank_tf.astype('int64')
    tf_idf.rank_idf=tf_idf.rank_idf.astype('int64')

    return tf_idf


In [8]:
all=make_tfidf(s140['trimmed'])

In [9]:
tf_idf_vect=TfidfVectorizer(use_idf=True,stop_words=stopw)
tf_idf=tf_idf_vect.fit_transform(s140['trimmed'].tolist())

In [108]:
s140_pos = s140[(s140.target == 'positive')]
s140_neg = s140[(s140.target == 'negative')]
s140_neut = s140[s140.target == 'neutral']

In [124]:
pos=make_tfidf(s140_pos['trimmed'])
neg=make_tfidf(s140_neg['trimmed'])
neut=make_tfidf(s140_neut['trimmed'])

In [149]:
a=10
b=50
print(f'All documents top words\n{all.head(a)}')
print('='*b)
print(f'Positive documents top words\n{pos.head(a)}')
print('='*b)
print(f'Negative documents top words\n{neg.head(a)}')
print('='*b)
print(f'Neutral documents top words\n{neut.head(a)}')

All documents top words
                tf       idf  rank_tf  rank_idf
_number_  0.078189  0.029137        0         0
day       0.022910  0.013212        1         1
good      0.020175  0.011472        2         2
_link_    0.017899  0.010193        3         4
get       0.017192  0.009713        4         6
go        0.016301  0.009873        5         5
like      0.016169  0.009095        6         8
work      0.015640  0.010381        7         3
love      0.015129  0.008805        8        10
today     0.014996  0.009305        9         7
Positive documents top words
                tf       idf  rank_tf  rank_idf
_number_  0.078936  0.029125        0         0
good      0.027984  0.014872        1         1
_link_    0.024132  0.012931        2         3
day       0.024000  0.013735        3         2
love      0.022817  0.012247        4         4
thanks    0.017382  0.009025        5         6
like      0.015616  0.008702        6         9
lol       0.015239  0.008468       

## Differentiating words

In [223]:
def find_keywords(ar1,ar2,ar3,t=1000):
    """Compare ar1 to ar2 and ar3"""

    ar1_w=np.copy(ar1.index[:]).tolist()
    
    a=np.minimum(len(ar1),t)
    b=np.minimum(len(ar2),t)
    c=np.minimum(len(ar3),t)
    i=0
    
    while i < a:
        if (ar1.index[i] in ar2.index[:b]) or (ar1.index[i] in ar3.index[:c]):
            del ar1_w[i]
            a-=1
            #ar1_w.pop(i)
        else:
            i+=1
    
    return ar1_w

In [224]:
treshold=100
key_pos=find_keywords(pos, neg, neut, treshold)
key_neg=find_keywords(neg, pos, neut, treshold)
key_neut=find_keywords(neut, neg, pos, treshold)

In [225]:
a=20
b=50

print(f'Positive documents top words\n{key_pos[:a]}')
print('='*b)
print(f'Negative documents top words\n{key_neg[:a]}')
print('='*b)
print(f'Neutral documents top words\n{key_neut[:a]}')

Positive documents top words
['though', 'little', 'watch', 'ready', 'excited', 'always', 'sound', 'hi', 'school', 'made', 'pretty', 'long', 'lot', 'looking', 'ur', 'ya', 'hour', 'wow', 'cute', 'beautiful']
Negative documents top words
['soon', 'away', 'cold', 'life', 'little', 'left', 'wo', 'headache', 'guy', 'another', 'man', 'trying', 'great', 'omg', 'nothing', 'early', 'someone', 'wait', '_notascii_', 'poor']
Neutral documents top words
['saw', 'business', 'okay', 'com', 'sb', 'boy', 'store', 'food', 'like', 'ceo', 'ap', 'good', 'oh', 'old', 'fitness', 'need', 'battle', 'warner', 'read', 'breakfast']
